<a href="https://colab.research.google.com/github/upinder-sangha/Image-Captioning/blob/main/COMP_6341_CV_PROJECT_PHASE_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Phase 2: Training the model to generate captions

##Getting the test and train sets from phase 1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pickle

In [ ]:
# !cp drive/MyDrive/COMP_6341_CV_PROJECT_RES/features.pkl /content
# !cp drive/MyDrive/COMP_6341_CV_PROJECT_RES/captions.pkl /content

# captions = pickle.load(open("captions.pkl","rb"))
# features = pickle.load(open("features.pkl","rb"))

In [4]:
!cp drive/MyDrive/COMP_6341_CV_PROJECT_RES/train_features.pkl /content
!cp drive/MyDrive/COMP_6341_CV_PROJECT_RES/test_features.pkl /content
!cp drive/MyDrive/COMP_6341_CV_PROJECT_RES/train_captions.pkl /content
!cp drive/MyDrive/COMP_6341_CV_PROJECT_RES/test_captions.pkl /content

train_features = pickle.load(open("train_features.pkl","rb"))
test_features = pickle.load(open("test_features.pkl","rb"))
train_captions = pickle.load(open("train_captions.pkl","rb"))
test_captions = pickle.load(open("test_captions.pkl","rb"))

print("size of training set = ", len(train_captions))

size of training set =  7120


##Preprocessing the data to match the format of the model

###Defining the tokenizer

In [5]:
vocab_size = 6000

from keras.preprocessing.text import Tokenizer
# convert a dictionary of clean captions to a list of captions
def to_list(captions):
    list_of_all_captions = list()
    for key in captions.keys():
       [list_of_all_captions.append(caption) for caption in captions[key]]
    return list_of_all_captions
 
# fit a tokenizer given a caption
def create_tokenizer(captions):
    list_of_all_captions = to_list(captions)
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(list_of_all_captions)
    return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(train_captions)
print('Number of unique words: %d' % (len(tokenizer.word_index)))
print('Vocab Size: %d' % vocab_size)


Number of unique words: 7955
Vocab Size: 6000


In [6]:
#save the tokenizer for next phase
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
!cp tokenizer.pkl drive/MyDrive/COMP_6341_CV_PROJECT_RES/

###finding and saving the maximum length of any caption in training data

In [7]:
# calculate the length of the description with the most words
def max_length(captions):
    list_of_all_captions = to_list(captions)
    return max(len(caption.split()) for caption in list_of_all_captions)

In [8]:
max_length = max_length(train_captions)
print(max_length)

pickle.dump(max_length, open('max_length.pkl', 'wb'))
!cp max_length.pkl drive/MyDrive/COMP_6341_CV_PROJECT_RES/

22


In [9]:
# Create inverse index mapping numbers to words
index_to_word = {v: k for k, v in tokenizer.word_index.items()}

###Mapping the words to their 50 dimentional glove word embeddings 

In [14]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
%ls

--2023-04-12 02:51:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-12 02:51:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-12 02:51:11--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [15]:
#This piece of code was referenced from an article "How to Use Word Embedding Layers for Deep Learning with Keras" 
#written by  Jason Brownlee on October 4, 2017 in Deep Learning for Natural Language Processing. https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

import os
import numpy as np

embeddings = {} # We create a dictionary of word -> embedding

with open("glove.6B.50d.txt") as f:
    for line in f:
        values = line.split()
        word = values[0] # The first value is the word, the rest are the values of the embedding
        embedding = np.asarray(values[1:], dtype='float32') # Load embedding
        embeddings[word] = embedding # Add embedding to our embedding dictionary

print('Found {:,} word vectors in GloVe.'.format(len(embeddings)))

Found 400,000 word vectors in GloVe.


In [16]:
print (embeddings['frog'])
print (len(embeddings['frog']))
print(type(embeddings['frog']))

[ 0.61038  -0.20757  -0.71951   0.89304   0.32482   0.76564   0.1814
 -0.33086   0.79173  -0.31664   0.011143  0.45412   1.5992    0.013494
 -0.093646  0.19245   0.251     1.1277   -1.0897   -0.42909  -1.1327
 -0.90465   0.5617   -0.058464  1.0007   -0.39017  -0.41665   0.73721
 -0.53824  -0.95993   0.67929  -0.59053   0.13408   0.54273  -0.36615
  0.014978 -0.2496   -0.81088   0.078905 -0.97552  -0.66394  -0.18508
 -0.87174   0.30782   1.2839   -0.14884   0.62178  -1.509     0.14582
 -0.31682 ]
50
<class 'numpy.ndarray'>


In [ ]:
print(tokenizer.word_index)

In [18]:
#This piece of code was referenced from an article "How to Use Word Embedding Layers for Deep Learning with Keras" 
#written by  Jason Brownlee on October 4, 2017 in Deep Learning for Natural Language Processing. https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

embedding_dim = 50 # We use 50 dimensional glove vectors

word_index = tokenizer.word_index
nb_words = min(vocab_size, len(word_index)) # How many words are there actually

embedding_matrix = np.zeros((nb_words, embedding_dim))

# The vectors need to be in the same position as their index. 

# Loop over all words in the word index
for word, i in word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= vocab_size: 
        continue
    # Get the embedding vector for the word
    embedding_vector = embeddings.get(word)
    # If there is an embedding vector, put it in the embedding matrix
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [ ]:
print("word at index 5: ",index_to_word.get(5))
print("embedding at index 5: \n",embedding_matrix[5])

word at index 5:  the
embedding at index 5: 
 [ 4.18000013e-01  2.49679998e-01 -4.12420005e-01  1.21699996e-01
  3.45270008e-01 -4.44569997e-02 -4.96879995e-01 -1.78619996e-01
 -6.60229998e-04 -6.56599998e-01  2.78430015e-01 -1.47670001e-01
 -5.56770027e-01  1.46579996e-01 -9.50950012e-03  1.16579998e-02
  1.02040000e-01 -1.27920002e-01 -8.44299972e-01 -1.21809997e-01
 -1.68009996e-02 -3.32789987e-01 -1.55200005e-01 -2.31309995e-01
 -1.91809997e-01 -1.88230002e+00 -7.67459989e-01  9.90509987e-02
 -4.21249986e-01 -1.95260003e-01  4.00710011e+00 -1.85939997e-01
 -5.22870004e-01 -3.16810012e-01  5.92130003e-04  7.44489999e-03
  1.77780002e-01 -1.58969998e-01  1.20409997e-02 -5.42230010e-02
 -2.98709989e-01 -1.57490000e-01 -3.47579986e-01 -4.56370004e-02
 -4.42510009e-01  1.87849998e-01  2.78489990e-03 -1.84110001e-01
 -1.15139998e-01 -7.85809994e-01]


###Splitting the training set into train and validation set

In [10]:
import random
def train_val_split(train_plus_val_captions,train_plus_val_features):

    train_plus_val_id_set = set(train_plus_val_captions.keys())
    train_validation_ratio = 0.9

    num_of_train = int(len(train_plus_val_id_set) * train_validation_ratio)

    # Randomly select elements for the train set
    train_id_set = set(random.sample(train_plus_val_id_set, num_of_train))

    # Create the validation set by removing the elements in the train set
    val_id_set = train_plus_val_id_set - train_id_set

    train_features = {k: train_plus_val_features[k] for k in train_id_set}
    val_features = {k: train_plus_val_features[k] for k in val_id_set}
    train_captions = {k: train_plus_val_captions[k] for k in train_id_set}
    val_captions = {k: train_plus_val_captions[k] for k in val_id_set}

    return train_captions, train_features, val_captions, val_features

In [11]:
train_captions, train_features, val_captions, val_features = train_val_split(train_captions,train_features)
print(len(train_captions),len(val_captions))

6408 712


<ipython-input-10-7c5632767f5b>:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_id_set = set(random.sample(train_plus_val_id_set, num_of_train))


##Defining our deeplearning model

In [12]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import add
from keras.layers import concatenate
from keras.layers import Reshape
from keras.callbacks import ModelCheckpoint
from keras.layers import SpatialDropout1D

In [19]:
# define the captioning model
def define_model4_1(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(2048,))
    image_layer_1 = Dropout(0.2)(inputs1)
    image_layer_2 = Dense(256, activation='relu')(image_layer_1)

    # sequence model
    inputs2 = Input(shape=(max_length,))
    text_layer_1 = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], trainable = False)(inputs2)
    text_layer_2 = Dense(256, activation='leaky_relu')(text_layer_1)

    # decoder model
    decoder1 = add([image_layer_2, text_layer_2])
    decoder2 = LSTM(256,dropout=0.5)(decoder1)
    decoder3 = Dense(516, activation='leaky_relu')(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)

    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

model4_1 = define_model4_1(vocab_size, max_length)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 dropout_1 (Dropout)            (None, 2048)         0           ['input_3[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 22, 50)       300000      ['input_4[0][0]']                
                                                                                              

###Defining the custom data generator class
It generates batches of data to train on for the model

In [ ]:
import tensorflow as tf
from keras.utils import pad_sequences
from keras.utils import to_categorical
import numpy as np
import random

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, captions, features, tokenizer, max_length, vocab_size, batch_Size, shuffle=True):
        
        self.captions = captions
        self.features = features
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.shuffle = shuffle
        self.batch_size = batch_Size
        self.keys = list(captions.keys())

        self.n = len(self.keys)

    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.keys)
    
    def __getitem__(self, index):
        
        start = index*self.batch_size
        end = (index+1)*self.batch_size
        batch_keys = self.keys[index*self.batch_size : (index+1)*self.batch_size]
        
        # X1,X2,y = create_sequences(batch_keys)
        X1, X2, y = list(), list(), list()

        for key in batch_keys:
            caption_list = self.captions[key]
            # walk through each caption for the image
            tokenized_caption_list = self.tokenizer.texts_to_sequences(caption_list)
            for tokenized_caption in tokenized_caption_list:
                # split one tokenized caption into multiple X,y pairs
                for i in range(1, len(tokenized_caption)):
                    # split into input and output pair
                    in_seq, out_word = tokenized_caption[:i], tokenized_caption[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                    # encode output sequence
                    out_word = to_categorical([out_word], num_classes=self.vocab_size)[0]
                    # out_word = embedding_matrix[out_word]
                    # store
                    X1.append(self.features[key][0])
                    X2.append(in_seq)
                    y.append(out_word)
        return [np.array(X1), np.array(X2)], np.array(y)
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
train_generator = CustomDataGen(train_captions, train_features,tokenizer, max_length,vocab_size,10)
validation_generator = CustomDataGen(val_captions, val_features,tokenizer, max_length,vocab_size,10)

In [ ]:
X, y= train_generator[0]
print(y)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


###Training the model

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

In [ ]:
filepath = 'model-ep{epoch:02d}.h5'
mcp = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False)
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
tb = TensorBoard('logs')

model4_1.fit(train_generator, epochs=50,callbacks=[es, rlr,mcp, tb], validation_data=validation_generator, verbose=1)

Epoch 1/50
639/640 [============================>.] - ETA: 0s - loss: 4.4248
Epoch 1: saving model to model-ep01.h5
640/640 [==============================] - 43s 53ms/step - loss: 4.4233 - val_loss: 3.7305 - lr: 0.0010
Epoch 2/50
640/640 [==============================] - ETA: 0s - loss: 3.4961
Epoch 2: saving model to model-ep02.h5
640/640 [==============================] - 32s 50ms/step - loss: 3.4961 - val_loss: 3.3962 - lr: 0.0010
Epoch 3/50
639/640 [============================>.] - ETA: 0s - loss: 3.2072
Epoch 3: saving model to model-ep03.h5
640/640 [==============================] - 32s 49ms/step - loss: 3.2074 - val_loss: 3.2582 - lr: 0.0010
Epoch 4/50
639/640 [============================>.] - ETA: 0s - loss: 3.0341
Epoch 4: saving model to model-ep04.h5
640/640 [==============================] - 32s 50ms/step - loss: 3.0336 - val_loss: 3.1784 - lr: 0.0010
Epoch 5/50
639/640 [============================>.] - ETA: 0s - loss: 2.9073
Epoch 5: saving model to model-ep05.h5
640/

###Saving models at different checkpoints to compare

In [ ]:
!cp model-ep07.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
!cp model-ep10.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/

In [ ]:
model4.save('my_model4_1.h5')

!cp my_model4_1.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
# !cp model-ep01.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
# !cp model-ep03.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
# !cp model-ep07.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
# !cp model-ep11.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
# !cp model-ep12.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/
# !cp model-ep13.h5 drive/MyDrive/COMP_6341_CV_PROJECT_RES/